In [8]:
import numpy as np
import jieqi_game as game

# Initialize the game from your FEN position
pos = game.Position.FromFen("2h3n2/1c3k3/b1a1p1b2/9/1Cp3p1N/p7R/7A1/2B6/P2CP4/4K1B1r w - - 6 46")
ph = game.PositionHistory()
ph.Reset(pos.GetBoard(), pos.GetRule50Ply(), pos.GetGamePly())

print("Starting position:")
print(f"Turn: {'Black' if pos.IsBlackToMove() else 'White'}")
print(f"Legal moves: {len(pos.GetBoard().GenerateLegalMoves())}")
print("=" * 50)

# Game loop
move_count = 0
max_moves = 20  # Limit for testing

while move_count < max_moves:
    current_pos = ph.Last()
    print(f"FEN: {game.GetFen(current_pos)}")
    
    # Check if game is over
    game_result = ph.ComputeGameResult()
    if game_result != game.GameResult.UNDECIDED:
        print(f"Game Over! Result: {game_result}")
        break
    
    # Check legal moves
    legal_moves = current_pos.GetBoard().GenerateLegalMoves()
    if not legal_moves:
        print("No legal moves!")
        break
    
    print(f"\nMove {move_count + 1}: {'Black' if current_pos.IsBlackToMove() else 'White'}")
    
    # MCTS search
    mcts = game.MCTS(ph, 20, 1.25)
    
    # Run 50 iterations (reduced for speed)
    for i in range(1000):
        enc = np.array(mcts.run_search_batch()).reshape(-1, 90, 167)
        length = enc.shape[0]
        
        # Dummy neural network outputs
        policy = np.ones((length, 2550)) / 2550
        value = np.zeros((length, 3))
        value[:, 1] = 1.0  # All draws
        
        mcts.apply_evaluations(policy, value)
    
    # Get move evaluations and select best move
    move_evals = list(mcts.get_root_move_evaluations())
    
    if not move_evals:
        print("No move evaluations!")
        break
    
    # Sort by visit count
    move_evals.sort(key=lambda x: x.visit_count, reverse=True)
    
    # Show top 3 moves
    print("Top moves:")
    for i, eval_info in enumerate(move_evals[:3]):
        print(f"  {i+1}. {eval_info.move.as_string()} - visits: {eval_info.visit_count} - WDL: {eval_info.win_prob},{eval_info.draw_prob},{eval_info.loss_prob}")
    
    # Play best move
    best_move = move_evals[0].move
    print(f"Playing: {best_move.as_string()}")
    
    ph.Append(best_move, game.PieceType.PIECE_TYPE_NB, game.PieceType.PIECE_TYPE_NB)
    move_count += 1

print(f"\nSimulation ended after {move_count} moves")

Starting position:
Turn: White
Legal moves: 44
FEN: 2h3n2/1c3k3/b1a1p1b2/9/1Cp3p1N/p7R/7A1/2B6/P2CP4/4K1B1r w - - 6 46

Move 1: White
Top moves:
  1. i5g6 - visits: 38000 - WDL: 0.9989736842105263,0.0010263157894736842,0.0
  2. e0d0 - visits: 39 - WDL: 0.0,1.0,0.0
  3. d1d0 - visits: 39 - WDL: 0.0,1.0,0.0
Playing: i5g6
FEN: 2h3n2/1c3k3/b1a1p1b2/6N2/1Cp3p2/p7R/7A1/2B6/P2CP4/4K1B1r b - - 7 46

Move 2: Black
Top moves:
  1. i9g9 - visits: 39467 - WDL: 0.0,0.0,1.0
  2. b1c1 - visits: 15 - WDL: 0.0,1.0,0.0
  3. b1b3 - visits: 15 - WDL: 0.0,1.0,0.0
Playing: i9g9
FEN: 2h3n2/1c3k3/b1a1p1b2/6N2/1Cp3p2/p7R/7A1/2B6/P2CP4/4K1r2 w - - 0 47
Game Over! Result: GameResult.BLACK_WON

Simulation ended after 2 moves
